In [1]:
import os


In [2]:
%pwd

'd:\\DataScience Projects\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DataScience Projects\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

The above is getting called through config.yaml file

In [9]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories


Calling configuration.py

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config    

In [14]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size


In [11]:
class DataIngestion:
    def __init__(
        self,config: DataIngestionConfig
    ):
        self.config = config
    
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    
    def unzip_data(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

Creating the pipeline

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()

except Exception as e:
    raise e
    
    

[2024-03-26 19:52:02,005]:  INFO: common: Read yaml file from config\config.yaml loaded successfully]
[2024-03-26 19:52:02,008]:  INFO: common: Read yaml file from params.yaml loaded successfully]
[2024-03-26 19:52:02,010]:  INFO: common: Created directory at: artifacts]
[2024-03-26 19:52:02,011]:  INFO: common: Created directory at: artifacts/data_ingestion]
[2024-03-26 19:52:06,769]:  INFO: 2947920949: artifacts/data_ingestion/data.zip dpwnload with following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "01ffe9a571502ad6a059d7f9b4c1e6488fc8de01c8a15ec25fc78308db400f70"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F794:2F9663:D36481:DC3B9A:66032762
Accept-Ranges: bytes
Date: Tue, 26 Mar 2024 19:52:03 GMT
Via: 1.1 varnish
X-Served-By: c